In [4]:
import hashlib

def gen_root(seed):
    v_a = hashlib.sha3_256(seed).digest()
    t_a = hashlib.sha3_256(seed + 'flag'.encode()).digest()[0] % 2
    
    return v_a, t_a

In [5]:
hashlib.sha3_256('12345'.encode() + 'hello'.encode()).hexdigest()

'59df11da56f507dec1bad6533d2a6747d392ede94caa72918ab959c15a11450c'

In [6]:
gen_root('12345'.encode())

(b'}N>\xec\x80\x02g\x19c\x9e\xd4\xdb\xa6\x89\x16\xeb\x94\xc7\xa4\x9a\x05>\x05\xc8\xf9W\x8f\xe4\xe5\xa3\xd7\xea',
 1)

In [7]:
depth = 5
N = 2**depth
coord = randint(0, N)
coord

18

In [8]:
def path(num, d=depth):
    """
    Convert an integer to a binary array using bit manipulation.
    
    Args:
        num (int): The input integer (must be ≥ 0).
        d (int): The desired length of the binary array.
    
    Returns:
        list: Binary array (e.g., 5 with d=4 → `[0, 1, 0, 1]`).
    """
    if num < 0:
        raise ValueError("Input must be non-negative")
    if d < 0:
        raise ValueError("Length must be positive")
    if num >= (1 << d):  # Check if number is too large for d bits
        raise ValueError(f"{num} requires more than {d} bits")
    
    return [(num >> (d - 1 - i)) & 1 for i in range(d)]

In [9]:
p = path(coord)
p

[1, 0, 0, 1, 0]

In [10]:
# level_0
import secrets

seed_a = secrets.token_bytes(32)
v_a1, t_a1 = gen_root(seed_a)

seed_b = secrets.token_bytes(32)
v_b1, t_b1 = gen_root(seed_b)

In [11]:
print(t_a1, t_b1)

1 0


In [12]:
def gen_descendants(v, t, cw):
    v_left = hashlib.sha3_256(v + 'l1'.encode()).digest()
    t_left = hashlib.sha3_256(v + 'l2'.encode()).digest()[0] % 2

    v_right = hashlib.sha3_256(v + 'r1'.encode()).digest()
    t_right = hashlib.sha3_256(v + 'r2'.encode()).digest()[0] % 2

    if t==1:
        v_left = bytes([v_left[i] ^^ cw[0][i] for i in range(32)])
        t_left ^^= cw[1]
        
        v_right = bytes([v_right[i] ^^ cw[2][i] for i in range(32)])
        t_right ^^= cw[3]

    return v_left, t_left, v_right, t_right

In [13]:
v_a1_left, t_a1_left, v_a1_right, t_a1_right = gen_descendants(v_a1, 0, [])
v_b1_left, t_b1_left, v_b1_right, t_b1_right = gen_descendants(v_b1, 0, [])

In [14]:
print(t_a1_left, t_b1_left)
print(t_a1_right, t_b1_right)

1 1
0 0


In [15]:
def find_cw_at_ith_level(i, v_a_i, t_a_i, v_b_i, t_b_i, p):
    v_a1_left, t_a1_left, v_a1_right, t_a1_right = gen_descendants(v_a_i, 0, [])
    v_b1_left, t_b1_left, v_b1_right, t_b1_right = gen_descendants(v_b_i, 0, [])
    
    #if t_a_i != t_b_i:
    if True:
        if p[i] == 0:
            # will be added to left descendants
            cw0 = bytes([0]*32)
            cw1 = t_a1_left ^^ t_b1_left ^^ 1
            
            # will be added to right descendants
            cw2 = bytes([v_a1_right[i] ^^ v_b1_right[i] for i in range(32)])
            cw3 = t_a1_right ^^ t_b1_right
    
            cw = [cw0, cw1, cw2, cw3]
    
        if p[i] == 1:
            # will be added to left descendants
            cw0 = bytes([v_a1_left[i] ^^ v_b1_left[i] for i in range(32)])
            cw1 = t_a1_left ^^ t_b1_left
    
            # will be added to right descendants
            cw2 = bytes([0]*32)
            cw3 = t_a1_right ^^ t_b1_right ^^ 1
    
            cw = [cw0, cw1, cw2, cw3]
    
    return cw 

In [16]:
cw = find_cw_at_ith_level(0, v_a1, t_a1, v_b1, t_b1, p)
cw

[b'\x05\xc7\x19{\r3p\x94\x86\x1d\xd9\xab^\x81\x0c\x93\x13\xe5\xcf\r\xcbFX\t\xebk w\xbca\xbb\x00',
 0,
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 1]

In [17]:
v_a1_left_, t_a1_left_, v_a1_right_, t_a1_right_ = gen_descendants(v_a1, t_a1, cw)
v_b1_left_, t_b1_left_, v_b1_right_, t_b1_right_ = gen_descendants(v_b1, t_b1, cw)

In [18]:
print(f'{v_a1_left_}\n{t_a1_left_}\n{v_a1_right_}\n{t_a1_right_}')

b'\xea\x89-\xae\xf6\xbb\xcb\xaf\xc6\x93F\xff\xba6\xe3\xe7\xef\x8f\x91\xbf\xa9\xe1Nw\xa0`\xac8t\x99\xcd:'
1
b'\x8bQ\x81\x1835\x03\xa7\x87\x19\xa9;\xf7g\xa2\xf6X\xbcG\xca\\\x96\xa6\xb6\x84jP\xf8Hy\xccw'
1


In [19]:
print(f'{v_b1_left_}\n{t_b1_left_}\n{v_b1_right_}\n{t_b1_right_}')

b'\xea\x89-\xae\xf6\xbb\xcb\xaf\xc6\x93F\xff\xba6\xe3\xe7\xef\x8f\x91\xbf\xa9\xe1Nw\xa0`\xac8t\x99\xcd:'
1
b'.!\xd1c\xf8W\x9c5\xe6\xbf\xfa\xba5m\xda\x9c\x1b_4Q\xb8n\x94\x98~h\\\xaeg^\xcc\xb9'
0


In [20]:
seed_a = secrets.token_bytes(32)
v_a, t_a = gen_root(seed_a)

seed_b = secrets.token_bytes(32)
v_b, t_b = gen_root(seed_b)

while t_a ^^ t_b != 1:
    seed_b = secrets.token_bytes(32)
    v_b, t_b = gen_root(seed_b)    

CW = []

for i in range(depth):
    cw = find_cw_at_ith_level(i, v_a, t_a, v_b, t_b, p)
    CW.append(cw)

    v_a_left_, t_a_left_, v_a_right_, t_a_right_ = gen_descendants(v_a, t_a, cw)
    v_b_left_, t_b_left_, v_b_right_, t_b_right_ = gen_descendants(v_b, t_b, cw)

    if p[i] == 0:
        v_a, t_a, v_b, t_b = v_a_left_, t_a_left_, v_b_left_, t_b_left_
    else:
        v_a, t_a, v_b, t_b = v_a_right_, t_a_right_, v_b_right_, t_b_right_

CW

[[b'\x04D2\xab\x06B\x0f\xbf\xe6\xca\x0c\xff\xd7\x93^\xdbG\xa5~\xaaS;\x0b\x82\xd1\xe7\x1bS/\x14\xec\xa0',
  1,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  0],
 [b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  1,
  b'\x11yY\xd8f\xd6\x9d_\xaa\x84\x84\xc0\x9fI\xd4\x85\x83\x05o\xa8!\x02PW\r`JO\xa1\x07\xc0\xac',
  0],
 [b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  0,
  b'\xce\x17\xdcX\x8eHP\x84`d\xfeq\xbe\x8e^K\xf2m\xd0\xed\xfd\x86O;\x1d\x15\xbe\xd6\xd9V\xa4+',
  1],
 [b'\xe7\x91[\x8a\xb2\x9dQ_R=\xa7\xe5\xef2\xb2l\x91\xc4\x108\xb0[\x9f0\x1f\x08*[S\xf4\xba\xc8',
  0,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  1],
 [b'\x00\x00\x00\x00\x

In [21]:
def compute_at_i(seed, CW, i):
    v, t = gen_root(seed)
    p = path(i)
    
    for j in range(depth):
        v_left_, t_left_, v_right_, t_right_ = gen_descendants(v, t, CW[j])
        if p[j] == 0:
            v, t = v_left_, t_left_
        else:
            v, t = v_right_, t_right_
    
    return t 

In [22]:
print(f'i\ta\tb\ta ^^ b')
print()
for i in range(N):
    a = compute_at_i(seed_a, CW, i)
    b = compute_at_i(seed_b, CW, i)
    print(f'{i}\t{a}\t{b}\t{a ^^ b}')

i	a	b	a ^^ b

0	1	1	0
1	1	1	0
2	1	1	0
3	1	1	0
4	1	1	0
5	1	1	0
6	0	0	0
7	1	1	0
8	1	1	0
9	0	0	0
10	1	1	0
11	1	1	0
12	0	0	0
13	1	1	0
14	1	1	0
15	1	1	0
16	0	0	0
17	0	0	0
18	1	0	1
19	0	0	0
20	1	1	0
21	0	0	0
22	0	0	0
23	0	0	0
24	1	1	0
25	1	1	0
26	0	0	0
27	0	0	0
28	0	0	0
29	1	1	0
30	0	0	0
31	1	1	0
